# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [2]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [3]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [3]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [4]:
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [5]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [6]:
tmp = sales.groupby(['ProductID'], as_index=False)[['Qty','Amt']].sum()
tmp

,ProductID,Qty,Amt
0,p1001771,1055,3354827
1,p1002841,903,11011541
2,p1005621,906,2601703
3,p1005771,3963,7319963
4,p1005891,5194,10119037
...,...,...,...
57,p1246581,2180,2456346
58,p1255281,809,955633
59,p1256521,701,1241545
60,p1284851,2350,6437323


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [11]:
tmp.sort_values(by='Amt', ascending=False).head()

,ProductID,Qty,Amt
22,p1072601,4058,18129067
43,p1178011,1653,14078818
1,p1002841,903,11011541
4,p1005891,5194,10119037
46,p1194801,990,7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [12]:
customers.groupby(['Gender'], as_index=False)['CustomerID'].count()

,Gender,CustomerID
0,F,2060
1,M,183


### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [8]:
customers['Age']=2022-customers['BirthYear']+1
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear,Age
0,c328222,2014-09-25,F,1960,63
1,c281448,2013-06-18,F,1974,49
2,c038336,2003-10-10,F,1968,55
3,c084237,2007-03-09,F,1982,41
4,c162600,2010-06-14,F,1978,45


### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [9]:
bin=[-np.inf,30,40,50,60,70,np.inf]
label=['30미만','30대','40대','50대','60대','70이상']

customers['AgeGroup']=pd.cut(customers['Age'], bins=bin, right=False, labels=label)
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,c328222,2014-09-25,F,1960,63,60대
1,c281448,2013-06-18,F,1974,49,40대
2,c038336,2003-10-10,F,1968,55,50대
3,c084237,2007-03-09,F,1982,41,40대
4,c162600,2010-06-14,F,1978,45,40대


### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [30]:
customers.groupby(['AgeGroup'], as_index=False).count()

,AgeGroup,CustomerID,RegisterDate,Gender,BirthYear,Age
0,30미만,3,3,3,3,3
1,30대,133,133,133,133,133
2,40대,859,859,859,859,859
3,50대,827,827,827,827,827
4,60대,299,299,299,299,299
5,70이상,122,122,122,122,122


### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [10]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
ID=pd.merge(sales, products, on='ProductID', how='inner')
ID.head()


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1
1,137,4,2016-01-02,p1036481,2,2100,c280590,순두부,반찬류,두부,1
2,63,16,2016-01-03,p1036481,1,1050,c037915,순두부,반찬류,두부,1
3,135,3,2016-01-04,p1036481,3,3150,c100815,순두부,반찬류,두부,1
4,63,13,2016-01-06,p1036481,10,10500,c048405,순두부,반찬류,두부,1


In [12]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.

product = ID.groupby(['ProductName'], as_index=False)['Amt'].sum()
product.head()

,ProductName,Amt
0,감귤컵푸딩,677154
1,감자칩,3676518
2,고구마스틱,1620986
3,깻잎,2930359
4,낱개캔디,479197


In [13]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.

product.sort_values(by=['Amt'],ascending=False).head()

,ProductName,Amt
37,우유1000,18129067
23,사과_소,14078818
9,딸기_대,11011541
7,두부_대,10119037
10,딸기_소,7517664


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [14]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.
age=pd.merge(sales,customers, on='CustomerID', how='inner')
age.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,49,40대
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,49,40대
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,49,40대
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,49,40대
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,49,40대


In [15]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.

age1=age.groupby(['AgeGroup'], as_index=False)['Amt'].sum()
age1.head()

,AgeGroup,Amt
0,30미만,127712
1,30대,7311268
2,40대,75865355
3,50대,81033746
4,60대,21990389


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [16]:
# 1) 세 데이터프레임을 merge 
t1=pd.merge(sales,customers, on='CustomerID', how='inner')
t2=pd.merge(t1,products, on='ProductID', how='inner')
t2.head()


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,49,40대,순두부,반찬류,두부,1
1,197,5,2017-01-24,p1036481,1,1050,c150417,2010-03-03,F,1974,49,40대,순두부,반찬류,두부,1
2,81,5,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1
3,81,6,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1
4,23,13,2016-10-16,p1036481,1,1050,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1


In [24]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다

t3=t2['AgeGroup'].unique().sort_values() #연령대 추출
for i in t3:
    x=t2.loc[t2['AgeGroup']==i]   #각 연령대
    y=x['Amt'].sum()  #나이대별전체 매출액
    z=x.groupby(['Category'], as_index=True)['Amt'].sum()/y  #카테고리별 비율
    print(z)


Category
간식     0.080470
과일     0.063424
반찬류    0.232492
유제품    0.147433
채소     0.476181
Name: Amt, dtype: float64
Category
간식     0.069540
과일     0.244395
반찬류    0.131457
유제품    0.293043
채소     0.261564
Name: Amt, dtype: float64
Category
간식     0.075854
과일     0.263431
반찬류    0.137199
유제품    0.254303
채소     0.269214
Name: Amt, dtype: float64
Category
간식     0.069172
과일     0.253603
반찬류    0.164576
유제품    0.232153
채소     0.280496
Name: Amt, dtype: float64
Category
간식     0.037408
과일     0.227170
반찬류    0.232762
유제품    0.185747
채소     0.316914
Name: Amt, dtype: float64
Category
간식     0.025296
과일     0.284068
반찬류    0.271096
유제품    0.105055
채소     0.314486
Name: Amt, dtype: float64
